# Pre-Process Kyoto

## Import zone

In [1]:
import pandas as pd
from mtranslate import translate
import pickle
import json_lines
import numpy as np
import os

## function zone 

In [2]:
def save_obj(file, obj):
    with open(file, 'wb') as f:
        pickle.dump(obj, f)

def load_obj(file):
    with open(file, 'rb') as f:
        return pickle.load(f)

In [3]:
path = r"C:\Users\sociocom\Desktop\Crawler\tweet_location"

## Data zone

In [4]:
#path = r"C:\Users\sociocom\Desktop\Crawler\tweet_location\2"
#list_tweets = []
#for filename in os.listdir(path):
#    with open(path + "\\" + filename, "rb") as f:
#        print(path + "\\" + filename)
#        for tweet in json_lines.reader(f):
#            list_tweets.append(tweet)

In [507]:
list_tweets = []
with open(r"C:\Users\sociocom\Desktop\Crawler\tweet_location\filter-2018-06-17.jsonl", "rb") as f:
    for tweet in json_lines.reader(f):
        list_tweets.append(tweet)

In [508]:
list_tweets_je = []
list_places = []
list_users = []
list_coor = []
for tweet in list_tweets:
    if tweet['lang'] != None:
        if (tweet['lang'] in ['ja','en']) and (tweet['place']!=None) and (tweet['place']['country_code']=="JP"):
            list_users.append(tweet['user'])
            d={}
            if tweet['coordinates']!=None: 
                d['ex_long'] = tweet['coordinates']['coordinates'][0]            
                d['ex_lat'] = tweet['coordinates']['coordinates'][1]
                list_coor.append(d)
            else:
                d['ex_long'] = None
                d['ex_lat'] = None
                list_coor.append(d)
            p = {}
            if tweet['place']!=None:
                p['place'] = tweet['place']['name']
                if tweet['place']['bounding_box']!=None and tweet['place']['bounding_box']['coordinates']!=None:
                    p['lot_sw'] = str(tweet['place']['bounding_box']['coordinates'][0][0])[1:-1]
                    p['lot_ne'] = str(tweet['place']['bounding_box']['coordinates'][0][2])[1:-1]
                else:
                    p['lot_sw'] = None
                    p['lot_ne'] = None
                list_places.append(p)
            else:
                p['place'] = None
                p['lot_sw'] = None
                p['lot_ne'] = None
            list_tweets_je.append(tweet)

In [509]:
df_tweet = pd.DataFrame(list_tweets_je)
df_place = pd.DataFrame(list_places)
df_user = pd.DataFrame(list_users)
df_coor = pd.DataFrame(list_coor)

In [510]:
df_user.rename(columns={"id_str":"id_user_str"}, inplace=True)

In [511]:
tweet_df = df_tweet.filter(['id_str','text','created_at','in_reply_to_status_id_str','in_reply_to_user_id_str','in_reply_to_screen_name','lang'],axis=1)

In [512]:
tweets_kyoto = pd.concat([df_user['screen_name'], df_user['id_user_str'],tweet_df, df_coor, df_place],axis=1)

In [513]:
tweets_kyoto = tweets_kyoto[np.isfinite(tweets_kyoto['ex_lat'])]

In [514]:
tweets_kyoto_filter = tweets_kyoto[tweets_kyoto['ex_lat']<35.055205]
tweets_kyoto_filter = tweets_kyoto_filter[tweets_kyoto_filter['ex_lat']>34.955205]
tweets_kyoto_filter = tweets_kyoto_filter[tweets_kyoto_filter['ex_long']<135.795300]
tweets_kyoto_filter = tweets_kyoto_filter[tweets_kyoto_filter['ex_long']>135.675300]

In [515]:
len(tweets_kyoto_filter)

1305

In [516]:
save_obj(path + '\mura_kyoto_tweet47', tweets_kyoto_filter)

In [17]:
tweets_kyoto_filter = load_obj(path + '\mura_kyoto_tweet3')

In [391]:
tweets_kyoto_filter.reset_index(inplace=True, drop="index")

In [392]:
tweets_kyoto_filter.head()

,screen_name,id_user_str,id_str,text,created_at,in_reply_to_status_id_str,in_reply_to_user_id_str,in_reply_to_screen_name,lang,ex_lat,ex_long,lot_ne,lot_sw,place
0,kouki_R750,964407677387292673,1001479162161500166,I'm at Starbucks Coffee 京都リサーチパーク店 - @starbuck...,Tue May 29 15:03:33 +0000 2018,None,None,None,ja,34.995875,135.738674,"136.987745, 35.779334","134.252598, 33.433256",Kansai
1,mgn_kyoto,487389327,1001481861720256512,【24:00時点 22.6℃ 65％】\n28.5~20.9℃\n＝京都府京都市 #kyot...,Tue May 29 15:14:17 +0000 2018,None,None,None,ja,35.013333,135.731667,"135.779805, 35.018685","135.730513, 34.99342",京都市 中京区
2,amds_kyotoKyoto,482601329,1001483466972676096,"京都(京都府) 24:00の 気温 22.6℃ 東北東の風,1.8m/s 湿度 65%RH ...",Tue May 29 15:20:39 +0000 2018,None,None,None,ja,35.013333,135.731667,"135.779805, 35.018685","135.730513, 34.99342",京都市 中京区
3,Wish__AZ,2741161340,1001483834083414016,"I'm at ラーメン荘 地球規模で考えろ in 京都市, 京都府 https://t.co...",Tue May 29 15:22:07 +0000 2018,None,None,None,ja,34.945372,135.765441,"135.877331, 34.973724","135.710465, 34.870748",京都市 伏見区
4,nn_arl,67044796,1001486868150673409,"I'm at 近鉄 竹田駅 in Fushimi, 京都府 https://t.co/YFM...",Tue May 29 15:34:10 +0000 2018,None,None,None,ja,34.956265,135.756188,"135.774048, 34.981071","135.70897, 34.937016",京都市 南区


## combine all

In [517]:
df_kyoto = pd.DataFrame()

In [518]:
path_all = path + "\kyoto_June1"

In [519]:
list_df = []
for filename in os.listdir(path_all):
    with open(path_all + "\\" + filename, "rb") as f:
        in_path = path_all + "\\" + filename
        print(in_path)
        df = load_obj(in_path)
        df.reset_index(inplace=True, drop="index")
        list_df.append(df)

C:\Users\sociocom\Desktop\Crawler\tweet_location\kyoto_June1\mura_kyoto_tweet33
C:\Users\sociocom\Desktop\Crawler\tweet_location\kyoto_June1\mura_kyoto_tweet34
C:\Users\sociocom\Desktop\Crawler\tweet_location\kyoto_June1\mura_kyoto_tweet35
C:\Users\sociocom\Desktop\Crawler\tweet_location\kyoto_June1\mura_kyoto_tweet36
C:\Users\sociocom\Desktop\Crawler\tweet_location\kyoto_June1\mura_kyoto_tweet37
C:\Users\sociocom\Desktop\Crawler\tweet_location\kyoto_June1\mura_kyoto_tweet38
C:\Users\sociocom\Desktop\Crawler\tweet_location\kyoto_June1\mura_kyoto_tweet39
C:\Users\sociocom\Desktop\Crawler\tweet_location\kyoto_June1\mura_kyoto_tweet40
C:\Users\sociocom\Desktop\Crawler\tweet_location\kyoto_June1\mura_kyoto_tweet41
C:\Users\sociocom\Desktop\Crawler\tweet_location\kyoto_June1\mura_kyoto_tweet42
C:\Users\sociocom\Desktop\Crawler\tweet_location\kyoto_June1\mura_kyoto_tweet43
C:\Users\sociocom\Desktop\Crawler\tweet_location\kyoto_June1\mura_kyoto_tweet44
C:\Users\sociocom\Desktop\Crawler\tweet_

In [520]:
df_kyoto = pd.concat(list_df)

In [521]:
df_kyoto.reset_index(inplace=True, drop="index")

In [522]:
df_kyoto.head()

,screen_name,id_user_str,id_str,text,created_at,in_reply_to_status_id_str,in_reply_to_user_id_str,in_reply_to_screen_name,lang,ex_lat,ex_long,lot_ne,lot_sw,place
0,anti_satsuki,903659715921567744,1002203935950692352,"I'm at JR 京都駅 in 京都市, 京都府 https://t.co/7Zy4sD6K8W",Thu May 31 15:03:33 +0000 2018,None,None,None,ja,34.985353,135.758295,"135.778292, 35.000701","135.734405, 34.977134",京都市 下京区
1,syohe8027,2282624300,1002204274707886082,I'm at Starbucks Coffee 京都リサーチパーク店 - @starbuck...,Thu May 31 15:04:53 +0000 2018,None,None,None,ja,34.995875,135.738674,"136.987745, 35.779334","134.252598, 33.433256",近畿地方
2,Wish__AZ,2741161340,1002204735355674624,"I'm at ラーメン横綱 一乗寺店 in 京都市, 京都府 https://t.co/Ei...",Thu May 31 15:06:43 +0000 2018,None,None,None,ja,35.041635,135.782933,"135.860702, 35.32117","135.725501, 35.00634",左京区
3,mgn_kyoto,487389327,1002206619072004096,【24:00時点 17.5℃ 92％】\n21.3~16.3℃\n＝京都府京都市 #kyot...,Thu May 31 15:14:12 +0000 2018,None,None,None,ja,35.013333,135.731667,"135.779805, 35.018685","135.730513, 34.99342",京都市 中京区
4,amds_kyotoKyoto,482601329,1002208198625869825,"京都(京都府) 24:00の 気温 17.5℃ 西南西の風,0.9m/s 湿度 92%RH ...",Thu May 31 15:20:29 +0000 2018,None,None,None,ja,35.013333,135.731667,"135.779805, 35.018685","135.730513, 34.99342",京都市 中京区


In [523]:
len(df_kyoto)

12048

In [524]:
save_obj(path_all + "\\kyoto_all", df_kyoto)

In [525]:
df_kyoto = load_obj(path_all + "\\kyoto_all")

In [526]:
df_kyoto.head()

,screen_name,id_user_str,id_str,text,created_at,in_reply_to_status_id_str,in_reply_to_user_id_str,in_reply_to_screen_name,lang,ex_lat,ex_long,lot_ne,lot_sw,place
0,anti_satsuki,903659715921567744,1002203935950692352,"I'm at JR 京都駅 in 京都市, 京都府 https://t.co/7Zy4sD6K8W",Thu May 31 15:03:33 +0000 2018,None,None,None,ja,34.985353,135.758295,"135.778292, 35.000701","135.734405, 34.977134",京都市 下京区
1,syohe8027,2282624300,1002204274707886082,I'm at Starbucks Coffee 京都リサーチパーク店 - @starbuck...,Thu May 31 15:04:53 +0000 2018,None,None,None,ja,34.995875,135.738674,"136.987745, 35.779334","134.252598, 33.433256",近畿地方
2,Wish__AZ,2741161340,1002204735355674624,"I'm at ラーメン横綱 一乗寺店 in 京都市, 京都府 https://t.co/Ei...",Thu May 31 15:06:43 +0000 2018,None,None,None,ja,35.041635,135.782933,"135.860702, 35.32117","135.725501, 35.00634",左京区
3,mgn_kyoto,487389327,1002206619072004096,【24:00時点 17.5℃ 92％】\n21.3~16.3℃\n＝京都府京都市 #kyot...,Thu May 31 15:14:12 +0000 2018,None,None,None,ja,35.013333,135.731667,"135.779805, 35.018685","135.730513, 34.99342",京都市 中京区
4,amds_kyotoKyoto,482601329,1002208198625869825,"京都(京都府) 24:00の 気温 17.5℃ 西南西の風,0.9m/s 湿度 92%RH ...",Thu May 31 15:20:29 +0000 2018,None,None,None,ja,35.013333,135.731667,"135.779805, 35.018685","135.730513, 34.99342",京都市 中京区


In [ ]:
#df_kyoto["text"] = df_kyoto["text"].apply(lambda x: translate(x, "en")) 

In [ ]:
#save_obj(path + "\\kyoto\\kyoto_all_transText", df_kyoto)

In [ ]:
#df_kyoto["place"] = df_kyoto["place"].apply(lambda x: translate(x, "en")) 

In [ ]:
#save_obj(path + "\\kyoto\\kyoto_all_translated", df_kyoto)

## Combine tranlated files

In [5]:
df_k1 = load_obj(path+"\\kyoto_June1\\kyoto_all_trans_june1")
df_k2 = load_obj(path+"\\kyoto_may\\kyoto_all_trans_may")

In [6]:
df_k = pd.concat([df_k1,df_k2])

In [7]:
len(df_k)

41004

In [8]:
#save_obj(path+"\\kyoto_translated1", df_k)